In [592]:
!pip install docx2python

In [1033]:
from docx2python import docx2python
import pandas as pd
import numpy as np
import os

In [1053]:
path = "C:/Users/user/Desktop/עמיר/לימודים/הנדסת תעשייה וניהול+אחר/Python/"

---

##  This class add a key and a value to a dictionary

In [1054]:
class MyDictionary(dict):
    def __init__(self):
        self = dict()
        
    def add(self,key,value):
        self[key] = value
        
myDict_1 = MyDictionary()
myDict_2 = MyDictionary()

---

## This function turn the word text thats not in table and put it into a dataframe

In [1055]:
def put_into_tabel(file_path):
    from docx import Document
    document = Document(file_path)
    myLst = []
    for p in document.paragraphs:
        myLst.append(p.text)
    myLst = pd.DataFrame(myLst)
    myLst.columns = ["TEXT"]
    return myLst

---

## This turn the medical history section into a dataframe 

In [1056]:
def medical_history(doc_path):
    bent_list1 = put_into_tabel(doc_path)
    lst2 = bent_list1.iloc[np.where(bent_list1.TEXT == "היסטוריה רפואית")[0][0]:np.where(bent_list1.TEXT == "חדות ראיה")[0][0]]
    
    first_text_dict = {"Medical History-background":"","Medical History-Medication sensitivity":"","Medical History-obstetrical history":"","Medical History-family history":"",
       "Medical History-reason for visit":""}
    myList = []
    for i in lst2.TEXT:
        if i!="":
            myList.append(i)
    myList.remove("היסטוריה רפואית")

    counter = 0 
    for key in first_text_dict.keys():
        myDict_1.add(key,myList[counter][myList[counter].find(":")+1:])
        counter+=1 
    mdh_Dataframe = pd.DataFrame(myDict_1, index=[0])
    return mdh_Dataframe

---

## This turn the squinting assessment section into a dataframe 

In [1059]:
def squinting_assessment(DocPath):
    bent_list2 = put_into_tabel(DocPath)
    lst3 = bent_list2.iloc[np.where(bent_list2.TEXT == "הערכת פזילה")[0][0]:]
    
    second_text_dict = {"Squinting Assessment-eye movement":"","Squinting Assessment-strabismus":""
                ,"Squinting Assessment-depth vision":"","Near point of convergence":"","Summary":"",
                        "Treatment Recommendations":"","Recommendations for Review":""}
    text_list = []
    for i in lst3.TEXT:
        if i != "":
            text_list.append(i)
    text_list.remove("הערכת פזילה")

    counter = 0 
    for key in second_text_dict.keys():

        if key == "Squinting Assessment-depth vision" and "ראית עומק:" in str(text_list):
            myDict_2.add(key,text_list[counter][text_list[counter].find(":")+1:])
            counter+=1 
            print(key)

        elif key == "Near point of convergence" and "Near point of convergence:" in str(text_list):
            myDict_2.add(key,text_list[counter][text_list[counter].find(":")+1:])
            counter+=1 
            print(1)

        elif key == "Squinting Assessment-depth vision" and "ראית עומק:" not in text_list:
            myDict_2.add(key,"empty")

        elif key == "Near point of convergence" and "Near point of convergence:" not in str(text_list):
            myDict_2.add(key,"empty")

        else:
            myDict_2.add(key,text_list[counter][text_list[counter].find(":")+1:])
            counter+=1 
        
    SA_Dataframe = pd.DataFrame(myDict_2, index=[0])
    return SA_Dataframe

---

## This function takes the path of a directory and put it into a list (it takes only docx files)

In [1060]:
def getFilePathsListFromDir(directory):
    file_paths = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f) and filename[-4:] == "docx":
            file_paths.append(f)
    return file_paths

---

## This function takes the personal details and transform all the data into a one row

In [1061]:
def personal_details(df):
    data = {}
    for i in df.index:
        data[i]=df[1][i]
    for j in df.index:
        i+=1
        data[i]=df[3][j]
    data = pd.DataFrame(data)
    data.columns = ["Name","ID", "Age","Sex", "Phone 1", "Phone 2"] 
    return data

---

## This function takes the do_ene (דו עיני) details and transform all the data into a one row

In [1062]:
def do_ene(df):
    df = df.fillna(value='[]')
    data_doEne = {}
    df.index = df.index + 1
    df_out = df.stack()
    df_out.index = df_out.index.map('{0[1]}_{0[0]}'.format)
    df_out = df_out.to_frame().T
    df_out=df_out.iloc[: , 4:]
    start = 0
    end = 1
    for k in range(len(df_out.columns)):
        if end % 3 == 0:
            start+=1
            end+=1
            continue
        else:
            answer = str(df_out.iloc[: , start:end]).split(" ")[-1]
            data_doEne[str(start)] = answer
            start+=1
            end+=1
    data_doEne = pd.DataFrame(data_doEne, index=[0])
    data_doEne.columns = ["far_without_repair_LEFT","far_without_repair_RIGHT",
                          "far_with_repair_LEFT","far_with_repair_RIGHT",
                          "close_without_repair_LEFT", "close_without_repair_RIGHT",
                          "close_with_repair_LEFT", "close_with_repair_RIGHT"] 
    return data_doEne

 ---

## The 3 functions below takes the משקפיים,רפרקציה ציקלופלגית ,רפרקציה סובייקטיבית details and changes the columns name.

In [1063]:
def glasses_function(glasses_data):
    glasses_data.columns = ["glasses_left_1","glasses_left_2", "glasses_left_3","glasses_left_4", "glasses_left_5",
                    "glasses_left_6",
                   "glasses_right_1","glasses_right_2","glasses_right_3","glasses_right_4","glasses_right_5",
                   "glasses_right_6",] 
    return glasses_data

In [1064]:
def cycloplegic_function(cycloplegic_data):
    cycloplegic_data.columns = ["cycloplegic_ref_left_1","cycloplegic_ref_left_2", "cycloplegic_ref_left_3",
                "cycloplegic_ref_left_4", "cycloplegic_ref_left_5","cycloplegic_ref_left_6",
               "cycloplegic_ref_right_1","cycloplegic_ref_right_2","cycloplegic_ref_right_3",
               "cycloplegic_ref_right_4","cycloplegic_ref_right_5","cycloplegic_ref_right_6",] 
    return cycloplegic_data

In [1065]:
def subjective_function(subjective_data):
    subjective_data.columns = ["subjective_ref_left_1","subjective_ref_left_2", "subjective_ref_left_3",
                "subjective_ref_left_4", "subjective_ref_left_5","subjective_ref_left_6",
               "subjective_ref_right_1","subjective_ref_right_2","subjective_ref_right_3",
               "subjective_ref_right_4","subjective_ref_right_5","subjective_ref_right_6",] 
    return subjective_data

---

## This function takes the בדיקת עיניים details and transform all the data into a one row

In [1066]:
def left_eye(data):
    myDict1 = {}
    for i in range(len(data.index)):
        myKey = data[0][0][0]+" "+data[1][i][0]
        myValue = data[0][i][0]
        myDict1[myKey] = myValue
    myDict1 = pd.DataFrame(myDict1, index=[0])
    return myDict1.iloc[:,1:]

In [1067]:
def right_eye(data):
    myDict2 = {}
    for i in range(len(data.index)):
        myKey = data[2][0][0]+" "+data[1][i][0]
        myValue = data[2][i][0]
        myDict2[myKey] = myValue
    myDict2 = pd.DataFrame(myDict2, index=[0])
    return myDict2.iloc[:,1:]

---

## <font color='red'>This function activate every thing</font>

In [1068]:
def active(path):
    doc_result = docx2python(path)
    personal_details_table = pd.DataFrame(doc_result.body[1][0:])
    do_ene_table=pd.DataFrame(doc_result.body[3][1:])
    glasses_data = pd.DataFrame(doc_result.body[5][1:])
    cycloplegic_data = pd.DataFrame(doc_result.body[7][1:])
    subjective_data = pd.DataFrame(doc_result.body[9][1:])
    left_right_eye_data = pd.DataFrame(doc_result.body[11][0:])
    all_files = [personal_details(personal_details_table),
           do_ene(do_ene_table),
           glasses_function(glasses_data),
           cycloplegic_function(cycloplegic_data),
           subjective_function(subjective_data),
           left_eye(left_right_eye_data),
           right_eye(left_right_eye_data),medical_history(path),squinting_assessment(path)]
    final_data = pd.concat(all_files,axis=1, ignore_index=False)
    return final_data

----

## Run thes two codes to save and the show the final data

In [1069]:
list_of_file = []
from tqdm import tqdm
loop=tqdm(total=len(getFilePathsListFromDir(path)),position=0,leave=False)
for z in range(len(getFilePathsListFromDir(path))):
    list_of_file.append(active(getFilePathsListFromDir(path)[z]))
    loop.set_description('Loading...'.format(z))
    loop.update(1)
final_df = pd.concat(list_of_file,axis=0, ignore_index=False)
print('Done!')
loop.close()

Loading...:  67%|██████▋   | 4/6 [00:01<00:00,  2.20it/s]

Squinting Assessment-depth vision
1


Loading...:  83%|████████▎ | 5/6 [00:02<00:00,  2.19it/s]

Squinting Assessment-depth vision
1


Squinting Assessment-depth vision
Done!


In [1070]:
final_df

,Name,ID,Age,Sex,Phone 1,Phone 2,far_without_repair_LEFT,far_without_repair_RIGHT,far_with_repair_LEFT,far_with_repair_RIGHT,...,Medical History-obstetrical history,Medical History-family history,Medical History-reason for visit,Squinting Assessment-eye movement,Squinting Assessment-strabismus,Squinting Assessment-depth vision,Near point of convergence,Summary,Treatment Recommendations,Recommendations for Review
0,ההההההה,555555555,6ש,נ,0528999999,,[F+F],[F+F],[],[],...,תקינה,רוחק ראיה וקוצר ראיה,בדיקה שגרתית בעבר דמעת שחלפה,מלאות,ללא עדות לפזילה,empty,empty,"ללא עדות לבעיה עינית, מיקוד תקין",ללא צורך בטיפול,לפי הצורך
0,רותי נבון,111111111,6ש,נ,0528999999,,[F+F],[F+F],[],[],...,תקינה,רוחק ראיה וקוצר ראיה,בדיקה שגרתית בעבר דמעת שחלפה,מלאות,ללא עדות לפזילה,empty,empty,"ללא עדות לבעיה עינית, מיקוד תקין",ללא צורך בטיפול,לפי הצורך
0,דדדדדדד,,12 חודשים,ז,0524444444,,[F+F],[F+F],[],[],...,תקינה,רוחק ראיה וקוצר ראיה,בדיקה לצורך שלילת גורמי סיכון לעין עצלה,מלאות,ללא עדות לפזילה,empty,empty,ללא עדות לבעיה עינית,ללא צורך בטיפול,עוד שנתיים
0,אאאאאאא,,8.6ש,נ,11111111,,[6/9],[6/6],[6/7.5],[],...,תקינה,אסטיגמטיזם לאב,הרכיבה מישקפיים ארבע שנים וחצי עקב אסטיגמצי...,מלאות,ללא עדות לפזילה,50 שניות של הקשת,תקין,היום כמעט ללא אסטיגמטיזם מיופיה קלה,אין צורך במישקפיים,עוד שנה
0,גגגגגגג,333333333,10.6 ש,נ,05233333333,,[],[],[6/6-],[6/6-],...,תקינה,מיופיה לאב,"ידוע על מיופיה, מרכיבה מישקפיים, מזה חודשיי...",מלאות,ללא עדות לפזילה,20 שניות של הקשת,תקין,מיופיה עם שינוי מינמלי,"המשך מישקפיים ללא שינוי , המשך טיפות של אטרופ...",עוד שנה
0,בבבבבבב,222222333,5.4ש,ז,0522222222,,[6/7.5],[6/7.5],[],[],...,תקינה,מיופיה ואסטיגמציה לאחות ולאב,בדיקה שגרתית לאחר השתלת מח עצם וטיפול בסטרו...,מלאות,ללא עדות לפזילה,100 שניות של הקשת,empty,"יובש עיני קל , עדשה שקופה וללא סימנים ללחץ ת...",טיפות FML פעמיים ביום לשבוע יחד עם הילוג'ל 3 ...,עוד שנה או קודם לפי הצורך


## Save it in excel and csv

In [750]:
final_df.to_excel(path+"EXCEL.xlsx")
final_df.to_csv(path+"CSV.csv")